In [5]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil
import pandas as pd

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('1655.T', 'yahoo', start, end)   #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)    #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)    #MAXIS ナスダック100上場投信
data = web.DataReader('2559.T', 'yahoo', start, end)      #MAXIS 全世界株式(オール・カントリー)上場投信
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               High      Low     Open    Close   Volume  Adj Close
Date                                                              
2020-01-07   9926.0   9926.0   9926.0   9926.0      0.0     9926.0
2020-01-08   9926.0   9926.0   9926.0   9926.0      0.0     9926.0
2020-01-09  10080.0  10080.0  10080.0  10080.0      0.0    10080.0
2020-01-10  10310.0  10280.0  10290.0  10310.0   1285.0    10310.0
2020-01-14  10400.0  10350.0  10390.0  10370.0   4223.0    10370.0
...             ...      ...      ...      ...      ...        ...
2022-09-12  14725.0  14635.0  14670.0  14710.0   8392.0    14710.0
2022-09-13  14845.0  14800.0  14845.0  14820.0   3573.0    14820.0
2022-09-14  14515.0  14375.0  14430.0  14390.0  12573.0    14390.0
2022-09-15  14410.0  14325.0  14405.0  14400.0   4748.0    14400.0
2022-09-16  14215.0  14145.0  14190.0  14200.0   8385.0    14200.0

[660 rows x 6 columns]


In [6]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [7]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**ボリンジャーバンドとRSI組み合わせ**

In [8]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   41.666667
Equity Final [$]                 11782542.024
Equity Peak [$]                 12087677.4805
Return [%]                           17.82542
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                    6.463403
Volatility (Ann.) [%]               17.568106
Sharpe Ratio                         0.367906
Sortino Ratio                         0.55111
Calmar Ratio                         0.214929
Max. Drawdown [%]                  -30.072278
Avg. Drawdown [%]                   -4.257961
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                      11.159605
Worst Trade [%]                     -6.616096
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1020,34,95,9798.2625,9150.0,-6.612278e+05,-0.066161,2020-02-27,2020-05-29,92 days
1,962,175,226,9697.7675,10780.0,1.041108e+06,0.111596,2020-09-25,2020-12-10,76 days
2,795,376,403,13054.3005,13400.0,2.748311e+05,0.026482,2021-07-21,2021-09-01,42 days
3,807,424,434,13194.9935,13770.0,4.640302e+05,0.043578,2021-10-04,2021-10-18,14 days
4,795,463,487,13978.8545,14420.0,3.507107e+05,0.031558,2021-11-30,2022-01-05,36 days
5,854,499,539,13416.0825,13970.0,4.730455e+05,0.041288,2022-01-24,2022-03-24,59 days
6,843,564,619,14159.7455,13970.0,-1.599555e+05,-0.013400,2022-04-28,2022-07-21,84 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   47.424242
Equity Final [$]               14774253.03925
Equity Peak [$]                14774253.03925
Return [%]                           47.74253
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   16.070083
Volatility (Ann.) [%]               19.447064
Sharpe Ratio                          0.82635
Sortino Ratio                          1.3749
Calmar Ratio                         0.534382
Max. Drawdown [%]                  -30.072278
Avg. Drawdown [%]                   -3.066687
Max. Drawdown Duration      252 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   14
Win Rate [%]                        92.857143
Best Trade [%]                       8.272342
Worst Trade [%]                     -7.942862
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1020,34,94,9798.26250,9020.0,-793827.75000,-0.079429,2020-02-27,2020-05-28,91 days
1,990,107,133,9295.78750,9790.0,489270.37500,0.053165,2020-06-16,2020-07-22,36 days
2,962,166,186,10069.59900,10330.0,250505.76200,0.025860,2020-09-10,2020-10-13,33 days
3,1025,198,207,9697.76750,10500.0,822288.31250,0.082723,2020-10-29,2020-11-12,14 days
4,859,328,339,12521.67700,12890.0,316389.45700,0.029415,2021-05-14,2021-05-31,17 days
5,850,355,364,13034.20150,13260.0,191928.72500,0.017324,2021-06-22,2021-07-05,13 days
6,878,375,391,12833.21150,13210.0,330820.30300,0.029360,2021-07-20,2021-08-16,27 days
7,883,396,403,13144.74600,13400.0,225389.28200,0.019419,2021-08-23,2021-09-01,9 days
8,908,417,434,13024.15200,13770.0,677229.98400,0.057267,2021-09-22,2021-10-18,26 days
9,894,463,487,13978.85450,14420.0,394384.07700,0.031558,2021-11-30,2022-01-05,36 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1020,34,95,9798.26250,9150.0,-661227.75000,-0.066161,2020-02-27,2020-05-29,92 days
1,-992,99,100,9413.17300,9440.0,-26612.38400,-0.002850,2020-06-04,2020-06-05,1 days
2,-964,101,102,9651.98500,9740.0,-84846.46000,-0.009119,2020-06-08,2020-06-09,1 days
3,-967,103,133,9532.57900,9790.0,-248926.10700,-0.027004,2020-06-10,2020-07-22,42 days
4,-903,145,146,9940.54950,10010.0,-62713.80150,-0.006987,2020-08-12,2020-08-13,1 days
5,-891,147,148,10000.25250,9990.0,9134.97750,0.001025,2020-08-14,2020-08-17,3 days
6,-875,158,161,10189.31200,10410.0,-193102.00000,-0.021659,2020-08-31,2020-09-03,3 days
7,900,175,226,9697.76750,10780.0,974009.25000,0.111596,2020-09-25,2020-12-10,76 days
8,-890,240,241,10895.79750,10970.0,-66040.22500,-0.006810,2020-12-30,2021-01-04,5 days
9,-888,242,245,10846.04500,11180.0,-296552.04000,-0.030790,2021-01-05,2021-01-08,3 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   72.121212
Equity Final [$]                9453582.82325
Equity Peak [$]                  13449224.578
Return [%]                          -5.464172
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -2.122634
Volatility (Ann.) [%]               15.972925
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -37.79743
Avg. Drawdown [%]                  -13.861214
Max. Drawdown Duration      907 days 00:00:00
Avg. Drawdown Duration      307 days 00:00:00
# Trades                                   42
Win Rate [%]                        23.809524
Best Trade [%]                      13.750037
Worst Trade [%]                     -4.174199
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-956,31,94,10457.97550,9020.0,1.374705e+06,0.137500,2020-02-21,2020-05-28,97 days
1,-1249,95,97,9104.70750,9100.0,5.879668e+03,0.000517,2020-05-29,2020-06-02,4 days
2,-1209,99,100,9413.17300,9440.0,-3.243384e+04,-0.002850,2020-06-04,2020-06-05,1 days
3,-1175,101,102,9651.98500,9740.0,-1.034176e+05,-0.009119,2020-06-08,2020-06-09,1 days
4,-1179,103,133,9532.57900,9790.0,-3.034994e+05,-0.027004,2020-06-10,2020-07-22,42 days
5,-1100,145,146,9940.54950,10010.0,-7.639555e+04,-0.006987,2020-08-12,2020-08-13,1 days
6,-1086,147,148,10000.25250,9990.0,1.113422e+04,0.001025,2020-08-14,2020-08-17,3 days
7,-1075,155,186,10109.70800,10330.0,-2.368139e+05,-0.021790,2020-08-26,2020-10-13,48 days
8,1105,199,207,9627.42100,10500.0,9.641998e+05,0.090635,2020-10-30,2020-11-12,13 days
9,-1099,210,215,10557.48050,10740.0,-2.005889e+05,-0.017288,2020-11-17,2020-11-25,8 days


<Strategy BBandRSI_WithShortPosition(upper_bound=65,lower_bound=30,rsi_window=10)>

**ボリンジャーバンドとRSI組み合わせ EntryRSI, ExitBolinger**

In [9]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.EntryRSI50andExitBB_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   54.545455
Equity Final [$]                10106155.8905
Equity Peak [$]                 12236442.6885
Return [%]                           1.061559
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                       0.404
Volatility (Ann.) [%]               13.072827
Sharpe Ratio                         0.030904
Sortino Ratio                        0.044085
Calmar Ratio                          0.01974
Max. Drawdown [%]                  -20.465723
Avg. Drawdown [%]                    -2.81261
Max. Drawdown Duration      494 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                   22
Win Rate [%]                        54.545455
Best Trade [%]                       8.392031
Worst Trade [%]                     -6.926851
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1184,65,95,8441.58000,9150.0,838769.28000,0.083920,2020-04-13,2020-05-29,46 days
1,-1177,107,133,9204.21250,9790.0,-689471.88750,-0.063643,2020-06-16,2020-07-22,36 days
2,1040,138,145,9758.06450,9990.0,241212.92000,0.023769,2020-07-31,2020-08-12,12 days
3,1052,165,175,9868.60900,9650.0,-229976.66800,-0.022152,2020-09-09,2020-09-25,16 days
4,-1031,180,200,9850.99500,9500.0,361875.84500,0.035630,2020-10-05,2020-11-02,28 days
5,1005,205,240,10461.52950,10950.0,490912.85250,0.046692,2020-11-10,2020-12-30,50 days
6,956,262,266,11516.72700,11730.0,203888.98800,0.018519,2021-02-03,2021-02-09,6 days
7,962,277,288,11657.42000,12210.0,531581.96000,0.047402,2021-02-26,2021-03-15,17 days
8,957,298,325,12270.43950,12650.0,363239.39850,0.030933,2021-03-29,2021-05-11,43 days
9,-976,328,339,12398.32300,12890.0,-479876.75200,-0.039657,2021-05-14,2021-05-31,17 days


**MACDとRSIの組み合わせ**

In [11]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   50.606061
Equity Final [$]                 6455560.9075
Equity Peak [$]                 10180197.0145
Return [%]                         -35.444391
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                  -15.388511
Volatility (Ann.) [%]                11.12441
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -41.133244
Avg. Drawdown [%]                   -21.21874
Max. Drawdown Duration      823 days 00:00:00
Avg. Drawdown Duration      415 days 00:00:00
# Trades                                   30
Win Rate [%]                        23.333333
Best Trade [%]                       5.164366
Worst Trade [%]                      -8.60578
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1062,99,100,9413.17300,9440.0,-28490.2740,-0.002850,2020-06-04,2020-06-05,1 days
1,-1033,101,102,9651.98500,9740.0,-90919.4950,-0.009119,2020-06-08,2020-06-09,1 days
2,-1036,103,104,9532.57900,9390.0,147711.8440,0.014957,2020-06-10,2020-06-11,1 days
3,-1121,106,136,8945.49950,9600.0,-733695.0605,-0.073165,2020-06-15,2020-07-29,44 days
4,-938,152,158,9900.74750,10240.0,-318218.8450,-0.034265,2020-08-21,2020-08-31,10 days
5,-866,161,162,10358.47050,10070.0,249815.4530,0.027849,2020-09-03,2020-09-04,1 days
6,-914,164,194,10089.80700,10040.0,45523.5980,0.004936,2020-09-08,2020-10-23,45 days
7,-864,226,227,10726.63900,10720.0,5736.0960,0.000619,2020-12-10,2020-12-11,1 days
8,-845,244,246,10975.40150,11290.0,-265835.7325,-0.028664,2021-01-07,2021-01-12,5 days
9,-799,247,248,11273.91650,11290.0,-12850.7165,-0.001427,2021-01-13,2021-01-14,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 549 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   43.030303
Equity Final [$]                11095325.3555
Equity Peak [$]                 12300438.2415
Return [%]                          10.953254
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                    4.048372
Volatility (Ann.) [%]               13.102964
Sharpe Ratio                         0.308966
Sortino Ratio                         0.51214
Calmar Ratio                         0.212355
Max. Drawdown [%]                  -19.064163
Avg. Drawdown [%]                    -4.29739
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration      110 days 00:00:00
# Trades                                   14
Win Rate [%]                             50.0
Best Trade [%]                        17.1352
Worst Trade [%]                     -4.790068
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1277,60,111,7828.56050,9170.0,1.713018e+06,0.171352,2020-04-06,2020-06-22,77 days
1,-1213,137,153,9651.98500,9980.0,-3.978822e+05,-0.033984,2020-07-30,2020-08-24,25 days
2,-1121,164,194,10089.80700,10040.0,5.583365e+04,0.004936,2020-09-08,2020-10-23,45 days
3,-1065,230,244,10676.88650,11030.0,-3.760659e+05,-0.033073,2020-12-16,2021-01-07,22 days
4,-975,247,259,11273.91650,11190.0,8.181859e+04,0.007443,2021-01-13,2021-01-29,16 days
5,-930,270,271,11900.79800,11990.0,-8.295786e+04,-0.007495,2021-02-16,2021-02-17,1 days
6,-928,272,278,11841.09500,11640.0,1.866162e+05,0.016983,2021-02-18,2021-03-01,11 days
7,-941,295,316,11880.89700,12450.0,-5.355259e+05,-0.047901,2021-03-24,2021-04-22,29 days
8,-839,354,368,12676.93700,12950.0,-2.290999e+05,-0.021540,2021-06-21,2021-07-09,18 days
9,-801,394,415,12995.35300,13270.0,-2.199922e+05,-0.021134,2021-08-19,2021-09-17,29 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1062,99,100,9413.17300,9440.0,-28490.2740,-0.002850,2020-06-04,2020-06-05,1 days
1,-1033,101,102,9651.98500,9740.0,-90919.4950,-0.009119,2020-06-08,2020-06-09,1 days
2,-1036,103,104,9532.57900,9390.0,147711.8440,0.014957,2020-06-10,2020-06-11,1 days
3,-1121,106,136,8945.49950,9600.0,-733695.0605,-0.073165,2020-06-15,2020-07-29,44 days
4,-938,152,158,9900.74750,10240.0,-318218.8450,-0.034265,2020-08-21,2020-08-31,10 days
5,-866,161,162,10358.47050,10070.0,249815.4530,0.027849,2020-09-03,2020-09-04,1 days
6,-914,164,194,10089.80700,10040.0,45523.5980,0.004936,2020-09-08,2020-10-23,45 days
7,-864,226,227,10726.63900,10720.0,5736.0960,0.000619,2020-12-10,2020-12-11,1 days
8,-845,244,246,10975.40150,11290.0,-265835.7325,-0.028664,2021-01-07,2021-01-12,5 days
9,-799,247,248,11273.91650,11290.0,-12850.7165,-0.001427,2021-01-13,2021-01-14,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 537 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   46.515152
Equity Final [$]                10383032.4755
Equity Peak [$]                 12206027.5475
Return [%]                           3.830325
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                    1.445522
Volatility (Ann.) [%]                13.15546
Sharpe Ratio                          0.10988
Sortino Ratio                        0.172325
Calmar Ratio                         0.076086
Max. Drawdown [%]                  -18.998421
Avg. Drawdown [%]                   -4.233027
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                   14
Win Rate [%]                        42.857143
Best Trade [%]                      17.253988
Worst Trade [%]                     -7.064698
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1267,57,107,7888.85750,9250.0,1.724568e+06,0.172540,2020-04-01,2020-06-16,76 days
1,-1214,137,153,9651.98500,9980.0,-3.982102e+05,-0.033984,2020-07-30,2020-08-24,25 days
2,-1122,164,194,10089.80700,10040.0,5.588345e+04,0.004936,2020-09-08,2020-10-23,45 days
3,-1072,229,244,10617.18350,11030.0,-4.425393e+05,-0.038882,2020-12-15,2021-01-07,23 days
4,-970,247,258,11273.91650,11010.0,2.559990e+05,0.023409,2021-01-13,2021-01-28,15 days
5,-940,270,271,11900.79800,11990.0,-8.384988e+04,-0.007495,2021-02-16,2021-02-17,1 days
6,-938,272,276,11841.09500,11850.0,-8.352890e+03,-0.000752,2021-02-18,2021-02-25,7 days
7,-934,295,316,11880.89700,12450.0,-5.315422e+05,-0.047901,2021-03-24,2021-04-22,29 days
8,-833,354,368,12676.93700,12950.0,-2.274615e+05,-0.021540,2021-06-21,2021-07-09,18 days
9,-796,394,414,12995.35300,13370.0,-2.982190e+05,-0.028829,2021-08-19,2021-09-16,28 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=15,MACDlong=40,MACDsignal=10,upper_bound=70,lower_bound=40,rsi_window=22)>

**MACDとボリンジャーバンドの組み合わせ**

In [12]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   23.484848
Equity Final [$]                 8666014.5355
Equity Peak [$]                 10472527.0525
Return [%]                         -13.339855
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -5.319984
Volatility (Ann.) [%]                6.459731
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -19.44231
Avg. Drawdown [%]                   -11.05432
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration      418 days 00:00:00
# Trades                                   14
Win Rate [%]                        28.571429
Best Trade [%]                       2.214154
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1087,95,106,9195.2925,8990.0,-223152.9475,-0.022326,2020-05-29,2020-06-15,17 days
1,993,133,136,9838.4605,9600.0,-236791.2765,-0.024238,2020-07-22,2020-07-29,7 days
2,950,145,152,10039.4505,9950.0,-84977.9750,-0.008910,2020-08-12,2020-08-21,9 days
3,841,245,258,11235.3410,11010.0,-189511.7810,-0.020056,2021-01-08,2021-01-28,20 days
4,786,266,276,11788.0635,11850.0,48682.0890,0.005254,2021-02-09,2021-02-25,16 days
5,759,288,295,12270.4395,11940.0,-250803.5805,-0.026930,2021-03-15,2021-03-24,9 days
6,712,325,326,12712.6175,12520.0,-137143.6600,-0.015152,2021-05-11,2021-05-12,1 days
7,689,339,354,12953.8055,12740.0,-147311.9895,-0.016505,2021-05-31,2021-06-21,21 days
8,651,403,413,13466.3300,13320.0,-95260.8300,-0.010866,2021-09-01,2021-09-15,14 days
9,627,434,451,13838.1615,14060.0,139092.7395,0.016031,2021-10-18,2021-11-11,24 days


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   34.090909
Equity Final [$]                 10062473.262
Equity Peak [$]                 11337796.5775
Return [%]                           0.624733
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                    0.238075
Volatility (Ann.) [%]               10.076142
Sharpe Ratio                         0.023628
Sortino Ratio                         0.03329
Calmar Ratio                         0.019478
Max. Drawdown [%]                  -12.222485
Avg. Drawdown [%]                   -3.447075
Max. Drawdown Duration      599 days 00:00:00
Avg. Drawdown Duration       84 days 00:00:00
# Trades                                   10
Win Rate [%]                             50.0
Best Trade [%]                       8.968801
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1087,95,166,9195.2925,10020.0,896457.0525,0.089688,2020-05-29,2020-09-10,104 days
1,990,240,259,11004.2025,11190.0,183939.5250,0.016884,2020-12-30,2021-01-29,30 days
2,939,266,278,11788.0635,11640.0,-139031.6265,-0.012560,2021-02-09,2021-03-01,20 days
3,891,288,296,12270.4395,11930.0,-303331.5945,-0.027745,2021-03-15,2021-03-25,10 days
4,789,403,415,13466.3300,13270.0,-154904.3700,-0.014579,2021-09-01,2021-09-17,16 days
5,757,434,463,13838.1615,13910.0,54381.7445,0.005191,2021-10-18,2021-11-30,43 days
6,727,487,494,14491.3790,14005.0,-353597.5330,-0.033563,2022-01-05,2022-01-17,12 days
7,725,539,564,14039.1515,14090.0,36865.1625,0.003622,2022-03-24,2022-04-28,35 days
8,728,619,649,14039.1515,14125.0,62497.7080,0.006115,2022-07-21,2022-09-02,43 days
9,709,657,659,14501.4285,14190.0,-220802.8065,-0.021476,2022-09-14,2022-09-16,2 days


<Strategy MACDandBBD(MACDshort=20,MACDlong=70,MACDsignal=10)>

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   44.848485
Equity Final [$]                 7898532.6565
Equity Peak [$]                 10472527.0525
Return [%]                         -21.014673
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -8.613644
Volatility (Ann.) [%]               10.215306
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.695679
Avg. Drawdown [%]                  -16.181004
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration      418 days 00:00:00
# Trades                                   18
Win Rate [%]                        27.777778
Best Trade [%]                       5.164366
Worst Trade [%]                      -8.60578
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1087,95,106,9195.2925,8990.0,-223152.9475,-0.022326,2020-05-29,2020-06-15,17 days
1,993,133,136,9838.4605,9600.0,-236791.2765,-0.024238,2020-07-22,2020-07-29,7 days
2,950,145,152,10039.4505,9950.0,-84977.9750,-0.008910,2020-08-12,2020-08-21,9 days
3,-937,164,194,10089.8070,10040.0,46669.1590,0.004936,2020-09-08,2020-10-23,45 days
4,-890,227,244,10666.9360,11030.0,-323126.9600,-0.034036,2020-12-11,2021-01-07,27 days
5,816,245,258,11235.3410,11010.0,-183878.2560,-0.020056,2021-01-08,2021-01-28,20 days
6,763,266,276,11788.0635,11850.0,47257.5495,0.005254,2021-02-09,2021-02-25,16 days
7,736,288,295,12270.4395,11940.0,-243203.4720,-0.026930,2021-03-15,2021-03-24,9 days
8,-711,315,326,12368.4715,12520.0,-107736.7635,-0.012251,2021-04-21,2021-05-12,21 days
9,670,339,354,12953.8055,12740.0,-143249.6850,-0.016505,2021-05-31,2021-06-21,21 days


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                        55.0
Equity Final [$]                  8761373.468
Equity Peak [$]                 13147102.2475
Return [%]                         -12.386265
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -4.923536
Volatility (Ann.) [%]               13.749554
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -38.201658
Avg. Drawdown [%]                  -20.881608
Max. Drawdown Duration      907 days 00:00:00
Avg. Drawdown Duration      457 days 00:00:00
# Trades                                   18
Win Rate [%]                        33.333333
Best Trade [%]                       7.502336
Worst Trade [%]                     -9.896229
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-999,33,107,10000.25250,9250.0,749502.2475,0.075023,2020-02-26,2020-06-16,111 days
1,1092,133,137,9838.46050,9700.0,-151198.8660,-0.014073,2020-07-22,2020-07-30,8 days
2,1055,145,152,10039.45050,9950.0,-94370.2775,-0.008910,2020-08-12,2020-08-21,9 days
3,-1041,164,194,10089.80700,10040.0,51849.0870,0.004936,2020-09-08,2020-10-23,45 days
4,-989,227,244,10666.93600,11030.0,-359070.2960,-0.034036,2020-12-11,2021-01-07,27 days
5,907,245,258,11235.34100,11010.0,-204384.2870,-0.020056,2021-01-08,2021-01-28,20 days
6,847,266,276,11788.06350,11850.0,52460.2155,0.005254,2021-02-09,2021-02-25,16 days
7,818,288,295,12270.43950,11940.0,-270299.5110,-0.026930,2021-03-15,2021-03-24,9 days
8,-790,315,326,12368.47150,12520.0,-119707.5150,-0.012251,2021-04-21,2021-05-12,21 days
9,745,339,354,12953.80550,12740.0,-159285.0975,-0.016505,2021-05-31,2021-06-21,21 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10)>

SMA and RSI

In [13]:
import BacktestingStrategies.Strategy_SMAandRSI as btstSMAandRsi
importlib.reload(btstSMAandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMAandRsi.EntryRSIandExitSMA_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "SMA_short": range(10, 25, 5),\
    "SMA_long": range(10, 75, 10),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "constraint": lambda x: (x.SMA_short < x.SMA_long),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMAandRsi.SMAandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   47.424242
Equity Final [$]                7966533.35325
Equity Peak [$]                   10448022.25
Return [%]                         -20.334666
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -8.314036
Volatility (Ann.) [%]               14.857692
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -32.374009
Avg. Drawdown [%]                  -16.827673
Max. Drawdown Duration      925 days 00:00:00
Avg. Drawdown Duration      464 days 00:00:00
# Trades                                   12
Win Rate [%]                        33.333333
Best Trade [%]                      16.068319
Worst Trade [%]                    -18.396825
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1105,36,41,9044.55000,8607.0,-4.834927e+05,-0.048377,2020-03-02,2020-03-09,7 days
1,1169,42,42,8140.09500,8100.0,-4.687106e+04,-0.004926,2020-03-10,2020-03-10,0 days
2,1113,43,44,8501.87700,8046.5,-5.068346e+05,-0.053562,2020-03-11,2020-03-12,1 days
3,1230,45,45,7285.88750,7250.0,-4.414162e+04,-0.004926,2020-03-13,2020-03-13,0 days
4,1116,46,49,7989.35250,7134.5,-9.540154e+05,-0.106999,2020-03-16,2020-03-19,3 days
5,1132,50,62,7034.65000,8165.0,1.279556e+06,0.160683,2020-03-23,2020-04-08,16 days
6,-982,99,172,9413.17300,9870.0,-4.486041e+05,-0.048531,2020-06-04,2020-09-18,106 days
7,-819,226,335,10726.63900,12700.0,-1.616183e+06,-0.183968,2020-12-10,2021-05-25,166 days
8,-532,407,421,13482.92750,13180.0,1.611574e+05,0.022467,2021-09-07,2021-09-29,22 days
9,-530,437,466,13841.14550,13645.0,1.039571e+05,0.014171,2021-10-21,2021-12-03,43 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 831 configurations.
  output = _optimize_grid()
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.

Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   33.333333
Equity Final [$]                14372029.8445
Equity Peak [$]                 14891529.8445
Return [%]                          43.720298
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   14.853247
Volatility (Ann.) [%]               14.941295
Sharpe Ratio                         0.994107
Sortino Ratio                        1.811219
Calmar Ratio                         1.499414
Max. Drawdown [%]                   -9.906035
Avg. Drawdown [%]                   -2.899823
Max. Drawdown Duration      257 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                        9.93967
Worst Trade [%]                     -0.977055
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1395,51,53,7165.29350,7877.5,993528.06750,0.099397,2020-03-24,2020-03-26,2 days
1,1350,54,81,8140.09500,8730.0,796371.75000,0.072469,2020-03-27,2020-05-11,45 days
2,1215,175,182,9697.76750,9940.0,294312.48750,0.024978,2020-09-25,2020-10-07,12 days
3,1255,199,208,9627.42100,10400.0,969586.64500,0.080248,2020-10-30,2020-11-13,14 days
4,999,376,430,13054.30050,13300.0,245453.80050,0.018821,2021-07-21,2021-10-12,83 days
5,969,464,478,13717.56750,13585.0,-128457.90750,-0.009664,2021-12-01,2021-12-21,20 days
6,981,499,502,13416.08250,13285.0,-128591.93250,-0.009771,2022-01-24,2022-01-27,3 days
7,979,503,545,13315.58750,14240.0,904999.83750,0.069423,2022-01-28,2022-04-01,63 days
8,1025,569,578,13601.99825,13495.0,-109673.20625,-0.007866,2022-05-11,2022-05-24,13 days
9,1039,579,622,13310.56275,13825.0,534500.30275,0.038649,2022-05-25,2022-07-26,62 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1074,35,36,9303.71750,9000.0,326192.59500,0.032645,2020-02-28,2020-03-02,3 days
1,-1093,37,38,9443.02450,9090.0,385855.77850,0.037385,2020-03-03,2020-03-04,1 days
2,-1139,39,40,9403.22250,9300.0,117570.42750,0.010977,2020-03-05,2020-03-06,1 days
3,-1250,41,42,8656.93500,8100.0,696168.75000,0.064334,2020-03-09,2020-03-10,1 days
4,-1369,43,44,8418.12300,8100.0,435510.38700,0.037790,2020-03-11,2020-03-12,1 days
...,...,...,...,...,...,...,...,...,...,...
109,-339,614,615,13433.17500,13450.0,-5703.67500,-0.001252,2022-07-13,2022-07-14,1 days
110,-337,616,617,13487.90275,13670.0,-61366.77325,-0.013501,2022-07-15,2022-07-19,4 days
111,-310,638,640,14473.00225,14500.0,-8369.30250,-0.001865,2022-08-18,2022-08-22,4 days
112,-309,654,655,14468.02700,14670.0,-62409.65700,-0.013960,2022-09-09,2022-09-12,3 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 800 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-16 00:00:00
Duration                    983 days 00:00:00
Exposure Time [%]                   34.545455
Equity Final [$]                 7539461.0565
Equity Peak [$]                 10081057.1025
Return [%]                         -24.605389
Buy & Hold Return [%]               43.058634
Return (Ann.) [%]                   -10.22275
Volatility (Ann.) [%]                9.063038
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.211603
Avg. Drawdown [%]                  -25.211603
Max. Drawdown Duration      877 days 00:00:00
Avg. Drawdown Duration      877 days 00:00:00
# Trades                                   64
Win Rate [%]                          26.5625
Best Trade [%]                       6.596476
Worst Trade [%]                     -3.903507
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1189,71,72,8408.17250,8340.0,81057.10250,0.008108,2020-04-21,2020-04-22,1 days
1,-1220,73,74,8258.91500,8300.0,-50123.70000,-0.004975,2020-04-23,2020-04-24,1 days
2,-1192,75,76,8408.17250,8540.0,-157138.38000,-0.015678,2020-04-27,2020-04-28,1 days
3,-1143,77,78,8637.03400,8530.0,122339.86200,0.012392,2020-04-30,2020-05-01,1 days
4,-1201,79,80,8318.61800,8530.0,-253869.78200,-0.025411,2020-05-07,2020-05-08,1 days
...,...,...,...,...,...,...,...,...,...,...
59,-572,616,617,13487.90275,13670.0,-104159.62700,-0.013501,2022-07-15,2022-07-19,4 days
60,-547,618,619,13915.77425,13970.0,-29661.48525,-0.003897,2022-07-20,2022-07-21,1 days
61,-545,620,621,13905.82375,13790.0,63123.94375,0.008329,2022-07-22,2022-07-25,3 days
62,-555,622,623,13756.56625,13875.0,-65730.73125,-0.008609,2022-07-26,2022-07-27,1 days


<Strategy SMAandRSI_WithShortPosition(SMA_short=15,SMA_long=70,upper_bound=75,lower_bound=30,rsi_window=16)>

In [ ]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("CombinationStrategies.csv")